In [11]:
import docx
from rouge import Rouge
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
import bert_score


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.0.0 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "C:\Users\steve\AppData\Roaming\Python\Python312\site-packages\ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "C:\Users\steve\AppData\Roaming\Python\Python312\site-packages\traitlets\config\application.py", line 1075, in launch_instance
    app.start()
  File "C:\Users\steve\AppData\Roaming\Python\Python312\site-packages\ipykernel\kernelapp.py", line 739, in start
    self.io_lo

In [ ]:
def extract_text_from_docx(docx_path):
    doc = docx.Document(docx_path)
    full_text = []
    for para in doc.paragraphs:
        full_text.append(para.text)
    return '\n'.join(full_text)

In [25]:
def compute_rouge(generated, ground_truth):
    rouge = Rouge()
    scores = rouge.get_scores(generated, ground_truth, avg=True)
    return scores

def compute_cosine_similarity(doc1, doc2):
    vectorizer = TfidfVectorizer()
    vectors = vectorizer.fit_transform([doc1, doc2])
    return cosine_similarity(vectors[0:1], vectors[1:2])[0][0]

def compute_bleu(generated, ground_truth):
    reference = [ground_truth.split()]
    candidate = generated.split()
    smoothie = SmoothingFunction().method4
    score = sentence_bleu(reference, candidate, smoothing_function=smoothie)
    return score

def compute_bertscore(generated, ground_truth):
    P, R, F1 = bert_score.score([generated], [ground_truth], lang="en", verbose=True)
    return F1.mean().item()

In [26]:
generated_path = r'2_article_log\6055.HK_20241120_0020.txt'
ground_truth_path = r'4_ground_truth\6055.HK.docx'

generated_report = open(generated_path, "r").read()
ground_truth_report = extract_text_from_docx(ground_truth_path)

In [32]:
rouge_score = compute_rouge(generated_report, ground_truth_report)
print(f"rouge score: {rouge_score}")

rouge score: {'rouge-1': {'r': 0.21884057971014492, 'p': 0.2915057915057915, 'f': 0.24999999510136625}, 'rouge-2': {'r': 0.05405405405405406, 'p': 0.07805907172995781, 'f': 0.06387569650301268}, 'rouge-l': {'r': 0.21014492753623187, 'p': 0.2799227799227799, 'f': 0.24006622026692917}}


In [33]:
similarity = compute_cosine_similarity(generated_report, ground_truth_report)
print(f"Cosine Similarity: {similarity}")

Cosine Similarity: 0.7372871930270356


In [ ]:
bleu_score = compute_bleu(generated_report, ground_truth_report)
print(f"BLEU Score: {bleu_score}")

BLEU Score: 0.03859414664857544


In [ ]:
bertscore = compute_bertscore(generated_report, ground_truth_report)
print(f"BERTScore: {bertscore}")

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


calculating scores...
computing bert embedding.


100%|██████████| 1/1 [00:17<00:00, 17.53s/it]


computing greedy matching.


100%|██████████| 1/1 [00:00<00:00,  5.01it/s]


done in 17.74 seconds, 0.06 sentences/sec
BERTScore: 0.6975311636924744
